In [12]:
split = [[0,1],[2,3]]

n = 4 #total elements
k = 2 #elements per split

split = []
for i in range(n//k):
    split.append(range(i*k,(i+1)*k))
split.append(range(n-n%k,n))

circuit_num =0

entangles = []
j=0

if j%2==0:
    for i in range(n-1):
        entangles.append((i,i+1))
    entangles.append((0,n-1))
else:
    for i in range(n-1,0,-1):
        entangles.append((i-1,i))
    entangles.append((0,n-1))

entangles


[(0, 1), (1, 2), (2, 3), (0, 3)]

In [27]:
def theta_gate(qc,theta,i):
    assert len(theta) == 3, "theta incorrect size"
    qc.rx(theta[0],i)
    qc.ry(theta[1],i)
    qc.rz(theta[2],i)
    return 0


def split_Z_gates(qc, n, j, indices,dag):
    phi = (-1)**dag*(np.pi/2)
    B = 2 in indices
    if j%2 ==0:
        qc.cz(0,1)
        qc.p(phi ,1-B)
    else:
        qc.p(phi ,1)
        qc.cz(0,1-B)
    return qc
    
def split_Z_gates_alt(qc, n, j, dag):
    phi = (-1)**dag*(np.pi/2)
    if j%2 ==0:
        qc.cz(0,1)
        qc.p(phi ,1)
        qc.p(phi ,2)
        qc.cz(2,3)
    else:
        qc.cz(2,3)
        qc.p(phi ,2)
        qc.p(np.pi/2 ,1)
        qc.cz(0,1)
    return qc           

def W_4_split(qc,theta, n, l, indices,dag):
    # Gets 2 qubit circuit, then adds W gate
    Theta_matrix = np.reshape(theta, (n,l,3))[indices,:,:]
    for j in range(l):
        for i in range(2):
            theta_gate(qc,Theta_matrix[i,j,:],i)
        split_Z_gates(qc, n, j, indices,dag)        
    return qc

def W_4_split_alt(qc,theta, n, l,dag):
    # Gets 4 qubit circuit, then adds W gate
    Theta_matrix = np.reshape(theta, (n,l,3))
    for j in range(l):
        for i in range(n):
            theta_gate(qc,Theta_matrix[i,j,:],i)
        split_Z_gates_alt(qc, n, j, dag)        
    return qc

def make_dags(i,splits):
    dags = np.zeros(splits)
    j=0
    for e in bin(i)[2:]:
        dags[j] = e
        j+=1
    return dags

In [60]:
import qiskit as qk
import numpy as np
n = 4
d = 1
Theta = 2*np.pi*np.random.random(n*d*3)

indices = [0,1]
qc = qk.QuantumCircuit(2)
qc = W_4_split(qc,Theta, n, d,indices, 1)
qc.draw()

# qc1 = qk.QuantumCircuit(n)
# qc1 = W_4_split_alt(qc1,Theta, n, d, 0)
# qc1.draw()

┌────────────┐┌────────────┐┌────────────┐              
q_0: ┤ Rx(5.8342) ├┤ Ry(2.9341) ├┤ Rz(2.2718) ├─■────────────
     ├────────────┤├────────────┤├────────────┤ │ ┌─────────┐
q_1: ┤ Rx(4.7696) ├┤ Ry(1.3808) ├┤ Rz(5.1104) ├─■─┤ P(-π/2) ├
     └────────────┘└────────────┘└────────────┘   └─────────┘

In [53]:
import qiskit as qk
import numpy as np
n = 4
d = 1
Theta = 2*np.pi*np.random.random(n*d*3)

splits = d*3
N = 2**(splits)
qcs = []

for i in range(N):
    qcs.append(qk.QuantumCircuit(n))
    dags = make_dags(i,splits)
    for j in range(splits):
        W_4_split_alt(qcs[i],Theta, n, d, dags[j])

In [70]:
import qiskit as qk
import numpy as np
n = 4
d = 1
Theta = 2*np.pi*np.random.random(n*d*3)

splits = d*4
N = 2**(splits)
qcs = []

for i in range(N):
    if i%2==0:
        indices = [0,1]
    else:
        indices = [2,3]
    qcs.append(qk.QuantumCircuit(2))
    dags = make_dags(i//2,splits-1)
    for j in range(splits-1):
        W_4_split(qcs[i],Theta, n, d, indices,dags[j])

In [79]:
qcs[13].draw()

┌────────────┐┌────────────┐┌────────────┐    ┌─────────┐  ┌────────────┐»
q_0: ┤ Rx(3.3428) ├┤ Ry(2.3145) ├┤ Rz(2.4515) ├─■──┤ P(-π/2) ├──┤ Rx(3.3428) ├»
     ├────────────┤├───────────┬┘├────────────┤ │ ┌┴─────────┴─┐├───────────┬┘»
q_1: ┤ Rx(5.9721) ├┤ Ry(2.659) ├─┤ Rz(1.4273) ├─■─┤ Rx(5.9721) ├┤ Ry(2.659) ├─»
     └────────────┘└───────────┘ └────────────┘   └────────────┘└───────────┘ »
«     ┌────────────┐┌────────────┐    ┌─────────┐  ┌────────────┐┌────────────┐»
«q_0: ┤ Ry(2.3145) ├┤ Rz(2.4515) ├─■──┤ P(-π/2) ├──┤ Rx(3.3428) ├┤ Ry(2.3145) ├»
«     ├────────────┤└────────────┘ │ ┌┴─────────┴─┐├───────────┬┘├────────────┤»
«q_1: ┤ Rz(1.4273) ├───────────────■─┤ Rx(5.9721) ├┤ Ry(2.659) ├─┤ Rz(1.4273) ├»
«     └────────────┘                 └────────────┘└───────────┘ └────────────┘»
«     ┌────────────┐   ┌────────┐
«q_0: ┤ Rz(2.4515) ├─■─┤ P(π/2) ├
«     └────────────┘ │ └────────┘
«q_1: ───────────────■───────────
«